# Random Forest Classifier

In [3]:
%run pipeline_utils.ipynb

(32560, 14)
(32560, 1)


In [4]:
from sklearn.ensemble import RandomForestClassifier
rfPipeline = Pipeline([
    ("preprocess", preprocess),
    ("rfc", RandomForestClassifier())
])

In [5]:
rfPipeline.fit(Xtr, Ytr.ravel())

Pipeline(memory=None,
         steps=[('preprocess',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('numerical_transform',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  FunctionTransformer(accept_sparse=False,
                                                                                      check_inverse=True,
                                                                                      func=<function select_number at 0x7f271a012ae8>,
                                                                                      inv_kw_args=None,
                                                                                      inverse_func=None,
                                                                                      kw_args=None,
                          

In [6]:
print("rfc score: ",rfPipeline.score(Xva,Yva))
print("auc training score: ", auc(rfPipeline,Xtr,Ytr))
print("auc validation score: ", auc(rfPipeline,Xva,Yva))

rfc score:  0.8385705483672212
[0 1 1 ... 0 0 1]
F1 weighted 0.9866189495960778
AUC : 0.9749161148102158
Classification report : 
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     22229
           1       0.99      0.95      0.97      7086

    accuracy                           0.99     29315
   macro avg       0.99      0.97      0.98     29315
weighted avg       0.99      0.99      0.99     29315

[[22171    58]
 [  337  6749]]
None
auc training score:  None
[0 0 1 ... 0 0 1]
F1 weighted 0.8437357607082104
AUC : 0.7457474723574404
Classification report : 
              precision    recall  f1-score   support

           0       0.88      0.92      0.90      2491
           1       0.68      0.57      0.62       755

    accuracy                           0.84      3246
   macro avg       0.78      0.75      0.76      3246
weighted avg       0.83      0.84      0.83      3246

[[2290  201]
 [ 323  432]]
None
auc validation score:  

### Hyper parameterization for RandomForest

In [7]:
# hyperparameters = {
#         'rfc__bootstrap': [True,False],
#         'rfc__max_depth': [10,25,50,75,None],
#         'rfc__max_features': ['auto','sqrt'],
#         'rfc__min_samples_leaf': [1,2,4],
#         'rfc__min_samples_split': [2,5,10],
#         'rfc__n_estimators':[200,500,1000,1500,2000]
    
#         }
# rfc = GridSearchCV(rfPipeline, hyperparameters, cv=10)
# rfc.fit(Xtr, Ytr.ravel())

In [8]:
#print(rfc.best_score_)

In [9]:
#print(rfc.best_estimator_)

In [10]:
#print(rfc.best_params_)

# Explicitly specify training and validation set for randomized grid search

In [11]:
Xsearch = pandas.concat((Xtr, Xva), axis=0)

In [12]:
Xsearch.shape

(32561, 14)

In [13]:
Ysearch = np.concatenate((Ytr, Yva), axis=0)

In [14]:
Ysearch.shape

(32561, 1)

In [15]:
XtrInd = np.full((Xtr.shape[0],), -1, dtype=int)

In [16]:
XtrInd.shape

(29315,)

In [17]:
XvaInd = np.full((Xva.shape[0],), 0, dtype=int)

In [18]:
XvaInd.shape

(3246,)

In [19]:
test_fold = np.append(XtrInd, XvaInd)

In [20]:
from sklearn.model_selection import PredefinedSplit

In [21]:
ps = PredefinedSplit(test_fold)

In [22]:
ps.get_n_splits()

1

In [24]:
# from sklearn.model_selection import GridSearchCV

# hyperparameters = {
#         'rfc__bootstrap': [True,False],
#         'rfc__max_depth': [10,50,75,None],
#         'rfc__max_features': ['auto','sqrt'],
#         'rfc__min_samples_leaf': [1,2,4],
#         'rfc__min_samples_split': [2,5,10],
# #         'rfc__n_estimators':[500,1000,2000]
    
#         }
# rfc = GridSearchCV(rfPipeline, hyperparameters, cv=ps)
# rfc.fit(Xsearch, Ysearch.ravel())

## Randomized Search for RandomForest

In [25]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
# parameters for GridSearchCV
# specify parameters and distributions to sample from
param_dist = {"rfc__max_depth": [3,4,5],
              "rfc__max_features": sp_randint(4, 7),
              "rfc__min_samples_split": sp_randint(7, 14),
              "rfc__min_samples_leaf": sp_randint(7, 14),
              "rfc__bootstrap": [True, False],
              "rfc__n_estimators":[1000,1500,2000]
             }
# run randomized search
n_iter_search = 5
random_search = RandomizedSearchCV(rfPipeline, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=ps)
random_search.fit(Xsearch, Ysearch.ravel())

RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
                   error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('preprocess',
                                              FeatureUnion(n_jobs=None,
                                                           transformer_list=[('numerical_transform',
                                                                              Pipeline(memory=None,
                                                                                       steps=[('selector',
                                                                                               FunctionTransformer(accept_sparse=False,
                                                                                                                   check_inverse=True,
                                                                                                           

In [26]:
#joblib.dump()
print(random_search.best_params_)

{'rfc__bootstrap': True, 'rfc__max_depth': 5, 'rfc__max_features': 6, 'rfc__min_samples_leaf': 8, 'rfc__min_samples_split': 12, 'rfc__n_estimators': 2000}


In [27]:
auc(rfPipeline,Xva,Yva)

[0 0 1 ... 0 0 1]
F1 weighted 0.8437357607082104
AUC : 0.7457474723574404
Classification report : 
              precision    recall  f1-score   support

           0       0.88      0.92      0.90      2491
           1       0.68      0.57      0.62       755

    accuracy                           0.84      3246
   macro avg       0.78      0.75      0.76      3246
weighted avg       0.83      0.84      0.83      3246

[[2290  201]
 [ 323  432]]
None


In [28]:
auc(rfPipeline,Xtr,Ytr)

[0 1 1 ... 0 0 1]
F1 weighted 0.9866189495960778
AUC : 0.9749161148102158
Classification report : 
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     22229
           1       0.99      0.95      0.97      7086

    accuracy                           0.99     29315
   macro avg       0.99      0.97      0.98     29315
weighted avg       0.99      0.99      0.99     29315

[[22171    58]
 [  337  6749]]
None


In [29]:
auc(rfPipeline,Xts,Yts)

[0 0 0 ... 0 0 1]
F1 weighted 0.9720319712535325
AUC : 0.9526746463691504
Classification report : 
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     24719
           1       0.97      0.92      0.94      7841

    accuracy                           0.97     32560
   macro avg       0.97      0.95      0.96     32560
weighted avg       0.97      0.97      0.97     32560

[[24460   259]
 [  660  7181]]
None


In [33]:
import joblib
joblib.dump(random_search.best_estimator_, 'rfc_grid_model.pkl')

['rfc_grid_model.pkl']